# Generación de texto usando GPT-2: Formas de decodificar la salida del texto generado

En este cuaderno se utilizá el modelo GPT-2 para generar texto. Se exploran diferentes formas de decodificar la salida del modelo para generar texto. Se verán los problemas que tienen cada una de las opciones, intentando en todos los casos generar texto que sea lo más natural y coherente posible.

Este cuaderno se ha desarrollado apoyándose en las explicaciones de [este post](https://huggingface.co/blog/how-to-generate) del blog de HuggingFace.

## Instalación e importación de librerías y definición de parámetros

Se instalan usando el comando `pip install` las librerías necesarias para el desarrollo del proyecto.

In [1]:
!pip install transformers==4.27.2
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 39.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
Looking in indexes: https://download.pytorch.org/whl/cu118


Se importan las librerías necesarias para este cuaderno.

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

Una vez importadas, se definen una serie de parámetros que se utilizarán a lo largo del cuaderno.

- **model_name**: Nombre del modelo que será reentrenado.
- **max_length**: Longitud máxima de las secuencias de texto generadas.

In [3]:
model_name = "gpt2"
max_length = 200

## Importación del modelo

Usando las clases Auto (más información sobre que es una clase Auto [aquí](https://huggingface.co/transformers/model_doc/auto.html)), se puede importar un modelo preentrenado de HuggingFace.

Se importa [GPT2](https://huggingface.co/transformers/model_doc/gpt2.html) y se define el tokenizer que se utilizará para procesar los datos. En este caso, se usa AutoModelForCasualLM, que es una clase que permite importar un modelo preentrenado para tareas de lenguaje natural. Más información sobre esta clase [aquí](https://huggingface.co/transformers/model_doc/auto.html#automodelforcausallm).

In [4]:
from transformers import GPT2Tokenizer, AutoModelForCausalLM

tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Dado que el modelo GPT-2 no tiene configurado el token de padding (el cual se utiliza para rellenar las secuencias de texto), se define el token de padding con el del token de fin de secuencia (EOS), cuyo valor es 50256.

In [5]:
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

## Métodos de decodificación

Dado un modelo de generación, existen difernetes formas de decodificar la salida producida por el modelo. El resultado que arroje el modelo dependerá de como de buena sea esa decodificación.

En esta sección se explicarán diferentes formas para realizar este proceso, estando todas ellas implementadas en la librería ```transformers```.

En este caso, como ya se ha comentado, se utilizará el modelo ```GPT2```, pero estas formas de decodificación son aplicables a otros conocidos modelos de generación como ```BART``` o ```T5```.

Para poder comparar los resultados que produzcan los difernetes métodos de generación, se utilizará el mismo texto de entrada al modelo, el cual se define a continuación.

In [6]:
inputText = "The University of Alicante is a"

### Greedy Search

En el método de búsqueda greedy (avaricioso), se selecciona la palabra con mayor probabilidad en cada paso. Este método es el más rápido, pero no siempre produce los mejores resultados.

Se muestra a continuación un ejemplo:

![Greedy Search](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/greedy_search.png)

El texto producido es el que está marcado en rojo (<span style="color:red">*The nice woman*</span>).

Como se puede ver, dada la palabra <span style="color:red">*The*</span> se producen tres posibles caminos, <span style="color:red">*dog*</span>, <span style="color:red">*nice*</span> y <span style="color:red">*car*</span>, con probabilidades 0.4, 0.5 y 0.1 respectivamente. Como 0.5 es la mayor de las probabilidades, se elige la palabra <span style="color:red">*nice*</span> y se continúa con el proceso. Del mismo modo se elige la palabra <span style="color:red">*woman*</span> frente a otras dos posiblidades.

En total, el camino elegido tiene una probabilidad de $0.5 \times 0.4 = 0.2$.

Visto un ejemplo, se procede ahora a generar texto usando este método.

In [7]:
input_ids = tokenizer.encode(inputText, return_tensors="pt")
output = model.generate(input_ids, max_length=max_length)
tokenizer.decode(output[0], skip_special_tokens=True)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


"The University of Alicante is a leading research university in the Netherlands. We are a member of the European Union's Scientific Council and the European Commission. We are also a member of the European Commission's Scientific Council.\n\nWe are a member of the European Commission's Scientific Council and the European Commission. We are also a member of the European Commission's Scientific Council. We are a member of the European Commission's Scientific Council. We are a member of the European Commission's Scientific Council. We are a member of the European Commission's Scientific Council. We are a member of the European Commission's Scientific Council. We are a member of the European Commission's Scientific Council. We are a member of the European Commission's Scientific Council. We are a member of the European Commission's Scientific Council. We are a member of the European Commission's Scientific Council. We are a member of the European Commission's Scientific Council. We are a m

Como se puede observar en la salida, el modelo comienza a producir texto con cierto sentido, pero a medida que se va generando texto, este va perdiendo sentido y empieza a repetirse, cosa que no nos interesa.

### Beam Search

Beam search (búsqueda de haz) es un método de búsqueda que se basa en la búsqueda greedy, pero en lugar de elegir la palabra con mayor probabilidad, se eligen las *k* palabras con mayor probabilidad. De estas *k* palabras, se elige la que tenga mayor probabilidad de producir una secuencia de palabras con sentido.

Para ello, en se definen el número de beams (```num_beams```), el cual indica el número de palabras que se seleccionarán en cada paso.

Se muestra a continuación un ejemplo:

![Beam search](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/beam_search.png)

El texto mostrado en la imagen es el mismo que el mostrado en el caso anterior. Ahora, si definimos que ```num_beams = 2``` podemos ver como, tras la palabra <span style="color:red">*The*</span>, se consideran dos caminos, los correspondientes a las palabras <span style="color:red">*nice*</span> y <span style="color:red">*dog*</span>, las cuales son las dos con mayor probabilidad.

En este caso, considerar el camino de las palabras <span style="color:red">*dog*</span> y <span style="color:red">*has*</span> tendría una probabilidad de $0.4 \times 0.9 = 0.34$, frente a la probabilidad de $0.2 del camino de las palabras <span style="color:red">*nice*</span> y <span style="color:red">*woman*</span>. Por tanto, se elige el primer camino al ser el que mayor probabilidad global tiene.

Visto el ejemplo teórico, se procede ahora a generar texto usando este método, definiendo el número de beams a 5 y estableciendo el parámetro ```early_stopping``` a ```True```, lo cual indica que el modelo dejará de generar texto cuando todos los beams hayan generado el token de final de secuencia (```<EOS>``` token).

In [8]:
input_ids = tokenizer(inputText, return_tensors="pt").input_ids
output = model.generate(input_ids, max_length=max_length, num_beams=5, early_stopping=True)
tokenizer.decode(output[0], skip_special_tokens=True)

'The University of Alicante is a member of the European Union, the European Commission, the European Parliament, the European Commission, the European Parliament, the European Parliament, the European Parliament, the European Parliament, the European Parliament, the European Parliament, the European Parliament, the European Parliament, the European Parliament, the European Parliament, the European Parliament, the European Parliament, the European Parliament, the European Parliament, the European Parliament, the European Parliament, the European Parliament, the European Parliament, the European Parliament, the European Parliament, the European Parliament, the European Parliament, the European Parliament, the European Parliament, the European Parliament, the European Parliament, the European Parliament, the European Parliament, the European Parliament, the European Parliament, the European Parliament, the European Parliament, the European Parliament, the European Parliament, the European

Podemos ver como en este caso el modelo se sigue repitiendo. Para intentar paliar esta sitación, se pueden utilizar métodos de penalización de repeticiones, como la penalización de n-gramas.

Esa penalización, la cual se puede establecer mediante el parámetro ```no_repeat_ngram_size```, indica el tamaño del n-grama que no se puede repetir, asignando un valor de probabilidad de 0 a las palabras que formen parte de un n-grama repetido.

Se prueba a continuación a generar texto usando este método, definiendo el parámetro ```no_repeat_ngram_size``` a 5.

In [9]:
input_ids = tokenizer(inputText, return_tensors="pt").input_ids
output = model.generate(input_ids, max_length=max_length, num_beams=5, early_stopping=True, no_repeat_ngram_size=5)
tokenizer.decode(output[0], skip_special_tokens=True)

'The University of Alicante is a member of the European Union, the European Commission, the European Parliament, the European Council, the European Parliament of the United Kingdom, and the European Parliament. It is a member state of the European Economic Area (EEA) of the European Union.'

Podemos ver como ahora mejora la respuesta que ofrece el modelo, no repitiéndose tanto como en el caso anterior.

Otro parámetro que se puede configurar es el ```num_return_sequences```, el cual indica el número de secuencias que se devolverán como salida. En el caso anterior, se devolvía una única secuencia, pero se puede devolver más de una.

Cabe destacar que siempre ```num_return_sequences``` deberá de ser menor o igual que ```num_beams```.

Se prueba a continuación retornando 3 secuencias.

In [10]:
input_ids = tokenizer(inputText, return_tensors="pt").input_ids
output = model.generate(input_ids, max_length=max_length, num_beams=5, early_stopping=True, no_repeat_ngram_size=5, num_return_sequences=3)


for i, seqOutput in enumerate(output):
    print(f"Sequence {i+1}: {tokenizer.decode(seqOutput, skip_special_tokens=True)}")
    print('*'*100)

Sequence 1: The University of Alicante is a member of the European Union, the European Commission, the European Parliament, the European Council, the European Parliament of the United Kingdom, and the European Parliament. It is a member state of the European Economic Area (EEA) of the European Union.
****************************************************************************************************
Sequence 2: The University of Alicante is a member of the European Union, the European Commission, the European Parliament, the European Council, the European Parliament of the United Kingdom, and the European Parliament. It is a member state of the European Economic Area (EEA), the European Economic Area of the European Union (EEA) and the EEA. It is also a member of the International Monetary Fund (IMF), the European Central Bank (ECB), the European Investment Bank (EIB) and the European Investment Bank for Reconstruction and Development (EIBRD). The University of Alicante was founded in 

Como se puede observar, los textos que devuelven los diferentes ```beams``` son basante parecidos. Esto se debe a que, como se ha comentado anteriormente, se eligen las *k* palabras con mayor probabilidad, por lo que es normal que los caminos que se sigan sean parecidos.

Continuamos probando con otros métodos de decodificación.

### Sampling

El método de sampling (muestreo) consiste en elegir la siguiente palabra de forma aleatoria, teniendo en cuenta la distribución de probabilidad de las palabras.

La distribución de probabilidad de las palabras se calcula usando la función softmax, la cual se define como:

$$w_t \sim P(w|w_{1:t-1})$$

Donde $w_t$ es la palabra que se va a elegir en el paso $t$ y $P(w|w_{1:t-1})$ es la distribución de probabilidad de las palabras.

Se muestra a continuación un ejemplo:

![vanilla_sampling](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/sampling_search.png)

En la imagen se puede observar la palabra de inicio, <span style="color:red">*The*</span>, y las tres posibles palabras que se pueden elegir en el siguiente paso, <span style="color:red">*nice*</span>, <span style="color:red">*dog*</span> y <span style="color:red">*car*</span>, con probabilidades 0.5, 0.4 y 0.1 respectivamente.

En este caso, se eliga la palabra <span style="color:red">*car*</span>, pero como se ha comentado anteriormente, la elección de la palabra se realiza de forma aleatoria, por lo que en otra ejecución podría haberse elegido otra palabra. Por tanto, este método se considera **no determinista**.

Del mismo modo es elegida la siguiente palabra, <span style="color:red">*drives*</span>.

Vista la teoría, se pasa a la práctica. Para usar el método de sampling, se define el parámetro ```do_sample``` a ```True```, desactivando el parámetro ```top_k``` (estableciéndolo a 0), el cual indica el número de palabras con mayor probabilidad que se considerarán en cada paso.

Además, mediante el parámetro ```random_seed``` se puede establecer una semilla para que los resultados sean reproducibles.

In [11]:
torch.manual_seed(0)

input_ids = tokenizer(inputText, return_tensors="pt").input_ids
output = model.generate(input_ids, max_length=max_length, do_sample=True, top_k=0)
tokenizer.decode(output[0], skip_special_tokens=True)

'The University of Alicante is a public university interested in contributing tourism to its university. The center also offers high schools. The center offers national homunculus programmes and pre and post education. Designing a Modern Fort by Emile Guichon is at the University of Alicante. Extensive Kettling of Clances at Itamar Hinner is at Sungloan University. Foreign languages at seed universities Libraries and Schools Dolingṇhatra Saltamran, Chandmirindran, Raesagesiyahu, Bungo, Kafirsana, Maersnu Sekhara Centre Bay of Faculty Education Sexual Wetlands Sea-born Kinja PK-10 : Manganakvasamyaksonhatana Pinghiknder Campanungson Basics of Training Centre Medicine Volton Root Training Centre Research Promoting Forestry Dale Park in the University of Alicante Salimar Majkodan Gandana Valley Vegetation Shoulders, Beanstock Shimelangill Tin Na Cob O'

En este caso, vemos como el texto producido no se repite. Ahora bien, conforme avanza la generación de texto, cada vez este tiene menos sentido.

Para intentar paliar esta situación, se puede establecer un parámetro llamado ```temperature```, el cual indica la temperatura de la distribución de probabilidad de las palabras. El *truco* consiste en incrementar la posibilidad de elegir palabras con mayor probabilidad, así como reducir la posibilidad de elegir palabras con menor probabilidad, haciendo que la distrubución de probabilidad $P(w|w_{1:t-1})$ sea más fuerte.

Esa temperatura se aplicará sobre la función softmax.

Se muestra a continuación el ejemplo anterior, pero usando una temperatura de 0.7:

![top_p_sampling](https://github.com/patrickvonplaten/scientific_images/blob/master/sampling_search_with_temp.png?raw=true)

Ahora podemos ver como, en el primer caso, la palabra <span style="color:red">*nice*</span>, la cual era la que tenía mayor probabilidad, ahora tiene aún mas probabilidad (pasa de 0.5 a 0.75). Por contra, la palabra <span style="color:red">*car*</span>, la cual tenía una probabilidad de 0.1, ahora tiene una probabilidad de 0.02.

Pasando a código, se prueba a generar texto usando este método, definiendo la temperatura a 0.7.

In [12]:
torch.manual_seed(0)

input_ids = tokenizer(inputText, return_tensors="pt").input_ids
output = model.generate(input_ids, max_length=max_length, do_sample=True, top_k=0, temperature=0.7)
tokenizer.decode(output[0], skip_special_tokens=True)

'The University of Alicante is a world renowned social research institute, providing a wide range of data to support the development of social research efforts.\n\nThe U.S. Department of State is a leading international research institute and an important player in the development of digital technologies.\n\nCzech Republic\n\nThe Czech Republic is an important partner in the region and contributes to the development of digital technologies.\n\nThe United States is another partner with the Czech Republic and is a key partner for the development of digital technologies.\n\nChina is a leading partner with the China National Science Foundation and is a key partner in the development of digital technologies.\n\nThe United Kingdom is a leading partner in the development of digital technologies and is a key partner for the development of digital technologies.\n\nMerkel is a key member of the European Union and is a key partner in the development of digital technologies.\n\nThe Netherlands is 

Dada la salida, podemos ver como en este caso tampoco tenemos repeticions, si bien el texto generado contiene varias frases (vease los diferentes carácteres no imprimibles *\n* que se generan), no teniendo mucha coherencia entre ellas.

¿Y que pasaría si se establece una temperatura a prácticamente 0? Se prueba a continuación.

In [13]:
torch.manual_seed(0)

input_ids = tokenizer(inputText, return_tensors="pt").input_ids
output = model.generate(input_ids, max_length=max_length, do_sample=True, top_k=0, temperature=0.01)
tokenizer.decode(output[0], skip_special_tokens=True)

'The University of Alicante is a leading research university in the world. We are a leading research university in the world. We are a leading research university in the world.\n\nWe are a leading research university in the world. We are a leading research university in the world.\n\nWe are a leading research university in the world. We are a leading research university in the world.\n\nWe are a leading research university in the world. We are a leading research university in the world.\n\nWe are a leading research university in the world. We are a leading research university in the world.\n\nWe are a leading research university in the world. We are a leading research university in the world.\n\nWe are a leading research university in the world. We are a leading research university in the world.\n\nWe are a leading research university in the world. We are a leading research university in the world.\n\nWe are a leading research university in the world.'

Se ve como los problemas de repetición vuelven a aparecer, convirtiendose este método en similar al greedy search.

### Top-K Sampling

En el método de top-k sampling, se eligen las *k* palabras con mayor probabilidad.

En la siguiente imagen se puede ver un ejemplo, en el que $K = 6$:

![top_k_sampling](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/top_k_sampling.png)

Se puede ver en la parte izquierda como se calcula la probabilidad para las posibles palabras que sucedan a *The*. En caso de elegir las 6 palabras con mayor probabilidad, el sumatorio de esa probabilida será de 0.68.

Bastante más acusado es el segundo caso, cuando se tiene que decidir que palabra va después de *car*. En este caso, el sumatorio de las probabilidades de las 6 palabras con mayor probabilidad es de 0.99.

Con este método se consiguen eliminar las palabras con menor probabilidad, las cuales suelen ser las que producen textos con menos sentido.

Viendo este método en código, se prueba a generar texto usando este método, definiendo el parámetro ```top_k``` a 50.

In [14]:
torch.manual_seed(0)

input_ids = tokenizer(inputText, return_tensors="pt").input_ids
output = model.generate(input_ids, max_length=max_length, do_sample=True, top_k=50)
tokenizer.decode(output[0], skip_special_tokens=True)

'The University of Alicante is a public university with an established campus, a high level of social responsibility, high achievement and a strong commitment to academic excellence.\n\nFaculty\n\nThe University of Alicante is a public university with an established campus, a high level of social responsibility, high achievement and a strong commitment to academic excellence.'

Podemos ver como el texto generado es bastante coherente. Por contra, podemos ver como al final el texto se repite.

Vemos a continuación un método que surgió a partir de este.

### Top-p sampling

Como se veía en el caso anterior, el método de top-k sampling muestreaba solo las *k* palabras con mayor probabilidad.

Por contra, ahora vemos el método de top-p sampling, el cual muestrea las palabras con mayor probabilidad hasta que la suma de las probabilidades de las palabras muestreadas sea mayor que un umbral dado.

Veamos un ejemplo:

![top_p_sampling](https://github.com/patrickvonplaten/scientific_images/blob/master/top_p_sampling.png?raw=true)

Aquí, el umbral se establece a $p = 0.92$, por tanto, el método escogerá palabras hasta que la suma de sus probabilidad llegue a 0.92.

Vemos que en el primer caso (el de la izquierda de la imagen), se eligen 9 palabras para poder alcanzar el umbral, mientras que en el segundo caso, eligiendo solo 3 se logra alcanzar el umbral.


Se prueba ahora con código a generar texto usando este método, definiendo el parámetro ```top_p``` a 0.92.

In [15]:
torch.manual_seed(0)

input_ids = tokenizer(inputText, return_tensors="pt").input_ids
output = model.generate(input_ids, max_length=max_length, do_sample=True, top_k=0, top_p=0.92)
tokenizer.decode(output[0], skip_special_tokens=True)

'The University of Alicante is a public university with research and tourism interests as well as national foundations. No grants are received in this program. Funds are provided by Student Assistants Ltd. Cyprus Design and Design Dept. by the Faculty of Design and worked within the Policy and Management Department. Research addresses domains of study identified by ASC: public relations, research and communications technology, environmental transport, environment, energy, electronic media and risk awareness. In practice, mainly bilateral collaborations and traditional economic and social analyses are carried out. The number of questions asked is a reflection of the demand for research in ecommerce to be provided by ACIA and universities. The research methodology and ability to successfully integrate relevant research topics is critical. Some findings regarding ecommerce consumption are clearly stated. First, we found that ecommerce consumption at the high cost of other public universit

Vemos que ahora no se producen repeticiones, si bien como pasaba anteriormente, el texto va perdiendo sentido a medida que se va generando.

¿Y si se combina este método Top-p con el método de Top-k? Se prueba a continuación. Así, se establece el parámetro ```top_k``` a 50 y el parámetro ```top_p``` a 0.92.

In [16]:
torch.manual_seed(0)

input_ids = tokenizer(inputText, return_tensors="pt").input_ids
output = model.generate(input_ids, max_length=max_length, do_sample=True, top_k=50, top_p=0.92)
tokenizer.decode(output[0], skip_special_tokens=True)

"The University of Alicante is a public university with an established campus, a high level of social responsibility, and a growing international presence. A proud member of the European Union and a member of the Organization of European Communities (OECD), Alicante is also a world leader in science, engineering, and mathematics.\n\nProfessor: It's not easy to imagine such a large-scale university with such an extensive base of alumni. Many of those who have gone on to earn their PhDs live here, while many in the other sectors have decided to relocate to the UK. What do you think this means for the UK?\n\nPete Wilschaft-Sebastien: Alicante has an alumni base that is large enough to meet demand and grow. But even with that the size of this university is not sufficient to meet demand in the academic workforce in general and the global workforce.\n\nProfessor: How did your university get to such an extent that it is attracting the most"

Comparando este texto con el anterior, ahora podemos ver que tiene bastante más sentido que el anterior.

Como vimos anteriormente, podemos generar varias secuencias de texto para probar diferentes caminos. Se prueba a continuación a generar 3 secuencias de texto.

In [17]:
torch.manual_seed(0)

input_ids = tokenizer(inputText, return_tensors="pt").input_ids
output = model.generate(input_ids, max_length=max_length, do_sample=True, top_k=50, top_p=0.92, num_return_sequences=3)

for i, seqOutput in enumerate(output):
    print(f"Sequence {i+1}: {tokenizer.decode(seqOutput, skip_special_tokens=True)}")
    print('*'*100)

Sequence 1: The University of Alicante is a public university with an established campus, a high level of social responsibility, and a growing international presence. A proud member of the European Union and a member of the Organization of European Communities (OECD), Alicante is also a world leader in science, engineering, and mathematics.

Professor: It's not easy to imagine such a large-scale university with such an extensive base of alumni. Many of those who have gone on to earn their PhDs live here, while many in the other sectors have decided to relocate to the UK. What do you think this means for the UK?

Pete Wilschaft-Sebastien: Alicante has an alumni base that is large enough to meet demand and grow. But even with that the size of this university is not sufficient to meet demand in the academic workforce in general and the global workforce.

Professor: How did your university get to such an extent that it is attracting the most
************************************************

Vemos como las primeras frases que genera el modelo en cada una de las secuencias tienen bastante coherencia, perdiéndose esta en las sucecivas frases.

# Conclusiones

A lo largo de este cuaderno se han visto diferentes formas de decodificar la salida de un modelo de generación de texto.

Hemos visto principalmente dos problemas que tienen estos métodos, que son la repetición de texto y la pérdida de sentido del texto generado.

Vistos todos ellos, podemos concluir que la combinación de los métodos de top-p sampling y top-k sampling es la que mejor resultados ofrece.

Muchas investigaciones se están llevando a cabo para mejorar estos métodos y conseguir así que los textos generados sean más coherentes y tengan más sentido.

¿Te ha gustado este cuaderno? Puedes ver en más detalle este problema en los siguientes *papers*:

- [Learning to Break the Loop: Analyzing and Mitigating Repetitions for Neural Text Generation](https://arxiv.org/pdf/2206.02369.pdf)
- [NEURAL TEXT DEGENERATION WITH UNLIKELIHOOD TRAINING](https://arxiv.org/pdf/1908.04319.pdf)
- [Consistency of a Recurrent Language Model With Respect to Incomplete Decoding](https://arxiv.org/pdf/2002.02492.pdf)